In [1]:
%load_ext autoreload
%autoreload 2


# Import Libraries

In [1]:
import os
from pathlib import Path

import torch
import pandas as pd
import gensim.downloader as api
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pack_sequence

from dataset import *
from model import *
from trainer import Trainer


# Read Data

In [2]:
path = "../data/"
train = pd.read_csv(os.path.join(path, "train.csv"))
test = pd.read_csv(os.path.join(path, "test.csv"))

train.head()


,movie_name,target,movie_description,id
0,Furies,0,Three furious vigilantes unite to take down a ...,133529636342330622371894152500993949030
1,RRR,0,The story of freedom fighters Komaram Bheem an...,133529660110779376651195430564179049830
2,John Wick,0,Legendary assassin John Wick (Keanu Reeves) re...,133529680710101630359923204885606137190
3,John Wick: Chapter 3 -- Parabellum,0,After gunning down a member of the High Table ...,133529687048354631501070212369122164070
4,Top Gun: Maverick,0,After more than thirty years of service as one...,133529699724860633783364227336154217830


# Create pretrained tokenizers

In [3]:
tok = Tokenizer()
tok_texts = [tok.tokenize(t) for t in train.movie_description.values]
vocab = Vocab(tok_texts, max_vocab_size=30000)


# Splitting Data

In [4]:
train_texts, train_labels, val_texts, val_labels = train_test_split(train)
train_dataset = TextDataset([tok.tokenize(t)
                            for t in train_texts], train_labels, vocab)
val_dataset = TextDataset([tok.tokenize(t)
                          for t in val_texts], val_labels, vocab)


# Create embeddings from tokens

In [5]:
os.environ["GENSIM_DATA_DIR"] = str(Path.cwd())
gensim_model = api.load("glove-wiki-gigaword-100")
emb_matrix = prepare_emb_matrix(gensim_model, vocab)


[==================================================] 100.0% 128.1/128.1MB downloaded


# Init Model and Config

In [6]:
config = {
    "freeze": False,
    "cell_type": "LSTM",
    "cell_dropout": 0.2,
    "num_layers": 2,
    "hidden_size": 128,
    "out_activation": "relu",
    "bidirectional": True,
    "out_dropout": 0.2,
    "out_sizes": [200],
}

trainer_config = {
    "lr": 3e-3,
    "n_epochs": 10,
    "weight_decay": 1e-6,
    "batch_size": 128,
    "device": "cuda" if torch.cuda.is_available() else "cpu"
}
clf_model = RecurrentClassifier(config, vocab, emb_matrix)


# Create Dataloaders and Train

In [7]:
train_dataloader = DataLoader(train_dataset,
                              batch_size=trainer_config["batch_size"],
                              shuffle=True,
                              num_workers=0,
                              collate_fn=train_dataset.collate_fn)
val_dataloader = DataLoader(val_dataset,
                            batch_size=trainer_config["batch_size"],
                            shuffle=False,
                            num_workers=0,
                            collate_fn=val_dataset.collate_fn)
t = Trainer(trainer_config)
t.fit(clf_model, train_dataloader, val_dataloader)


Epoch 1/10


  0%|          | 0/54 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 2/10


  0%|          | 0/54 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 10/10


  0%|          | 0/54 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

RecurrentClassifier(
  (embeddings): Embedding(30003, 100, padding_idx=0)
  (cell): LSTM(100, 128, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (out_dropout): Dropout(p=0.2, inplace=False)
  (out_proj): Sequential(
    (0): Linear(in_features=512, out_features=200, bias=True)
    (1): Linear(in_features=200, out_features=6, bias=True)
  )
)

# Save Model

In [8]:
t.save("baseline_model.ckpt")


# Load pretrained Model

In [9]:
t = Trainer.load("baseline_model.ckpt")


# Define predict function

In [10]:
def predict(model, text):
    tok_text = tok.tokenize(text)
    indexed_text = torch.tensor(vocab.vectorize(tok_text)).to(t.device)
    genre = model(pack_sequence([indexed_text])).argmax().item()
    return genre


# Get testset predictions

In [11]:
test_dataloader = DataLoader(TextDataset([tok.tokenize(t) for t in test.movie_description.values], [-1] * test.shape[0], vocab),
                             batch_size=trainer_config["batch_size"],
                             shuffle=False,
                             num_workers=0,
                             collate_fn=val_dataset.collate_fn)

predictions = t.predict(test_dataloader)


# Create submission

In [28]:
sample_submission = pd.read_csv(os.path.join(path, "sample_submission.csv"))
sample_submission["target"] = predictions
sample_submission.head()
print(sample_submission.shape)
t = sample_submission.drop_duplicates()
print(t.groupby(['id']).head())


(2684, 2)
                                           id  target
0     133529667241314002934985813983134580070       5
1     133529693386607632642217219852638190950       1
2     133529737754378640630246272237250379110       1
3     133529756769137644053687294687798459750       1
4     133529828866765532034234504812793265510       5
...                                       ...     ...
2679  133594735746623717950164639758666530150       5
2680  133594753176819471088318910338335604070       5
2681  133594873603626492770112052525140114790       4
2682  133594879149597868768615684073216638310       5
2683  133594920348242376186071232716070813030       5

[2684 rows x 2 columns]


In [13]:
sample_submission.to_csv("submission.csv", index=False)
